In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
os.listdir("/content/gdrive/MyDrive/archive")

In [ ]:
os.listdir("/content/gdrive/MyDrive/archive/skin/train_set")

In [ ]:
os.listdir("/content/gdrive/MyDrive/archive/skin/test_set")

In [ ]:
import tensorflow as tf
import os
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
train_dir="/content/gdrive/MyDrive/archive/skin/train_set"

In [ ]:
test__dir="/content/gdrive/MyDrive/archive/skin/test_set"

In [ ]:
label=["'malignant","benign"]

In [ ]:
print("class")
for i in range(len(label)):
  print(i,end=" ")
  print(label[i])

In [ ]:
print("number of classes:",len(label))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

In [ ]:
# Define data directories
train_dir ="/content/gdrive/MyDrive/archive/skin/train_set"
test_dir = "/content/gdrive/MyDrive/archive/skin/test_set"

In [ ]:
# Define data generators for training and testing data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2,)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='validation')
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:
# Load the ResNet50 model with pre-trained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# Set the path to the dataset
data_path ='/content/gdrive/MyDrive/archive/skin/test_set'

# Define the classes
classes = ['malignant', 'benign']

# Print 10 images of each class
for cls in classes:
    cls_path = os.path.join(data_path, cls)
    cls_images = os.listdir(cls_path)[:10]
    print(f"{cls.capitalize()} Images:")
    for img_file in cls_images:
        img_path = os.path.join(cls_path, img_file)
        img = plt.imread(img_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    #print('\n')


In [ ]:
# Add custom layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
#output layer fully connected dance layer with two neuron
predictions = Dense(2, activation='softmax')(x)

In [ ]:
import matplotlib.pyplot as plt

# Get the first 10 images from the test generator
x_test, y_test = next(test_generator)
x_test_first10 = x_test[:10]

# Plot the first 10 images
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_test_first10[i])
    ax.set_title("Image {}".format(i+1))
    ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Combine the base ResNet50 model with the custom layers
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

In [ ]:
# Freeze all layers in the base ResNet50 model
for layer in base_model.layers[5:]:
    layer.trainable = False
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model on the training data
history = model.fit_generator(train_generator, 
                              steps_per_epoch=train_generator.n // train_generator.batch_size, 
                              epochs=45, 
                              validation_data=val_generator, 
                              validation_steps=val_generator.n // val_generator.batch_size)

In [ ]:
# Plot the loss vs val_loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the accuracy vs val_accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('Test Accuracy:', test_acc)
# Print the train and test loss
print('Train Loss:', history.history['loss'][-1])
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
# Plot the train and validation loss over epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.show()


In [ ]:
# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('Test Accuracy:', test_acc)

# Plot the training and testing accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'val'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Get the next batch of images from the test generator
x_test, y_test = next(test_generator)

# Predict the classes of the testing data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
label_dict = {0: 'benign', 1: 'malignant'}

# Plot the images along with their predicted and actual labels
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12,12))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_test[i])
    pred_label = label_dict[y_pred_classes[i]]
    true_label = label_dict[y_true_classes[i]]
    ax.set_title("Pred: {}\nTrue: {}".format(pred_label, true_label))
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()